In [2]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma
import model_circular_economy as mc
import model_ippu as mi
import model_socioeconomic as ms

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(ms)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:719: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:719: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_socioeconomic' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_socioeconomic.py'>

In [6]:
df_emissions = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Emissions_Totals_E_All_Data_(Normalized)/Emissions_Totals_E_All_Data_(Normalized).csv")
df_item_codes_to_mb_sum_factor = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU/fao_emissions_mass_balance_equation_by_item_code.csv")


dict_ic_to_sf = sf.build_dict(df_item_codes_to_mb_sum_factor[["item_code", "sum_factor"]])

In [51]:
df_emissions[
    df_emissions["Year"].isin([2019]) & 
    df_emissions["Element"].isin(em_elems) &
    df_emissions["Item Code"].isin(dict_ic_to_sf.keys()) &
    df_emissions["Area"].isin(["Costa Rica"])
]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Source Code,Source,Unit,Value,Flag,Note
217597,48,Costa Rica,5058,Enteric Fermentation,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,2758.5586,Fc,NaN
217914,48,Costa Rica,5059,Manure Management,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,139.3946,Fc,NaN
218097,48,Costa Rica,5060,Rice Cultivation,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,24.8191,Fc,NaN
218402,48,Costa Rica,5061,Synthetic Fertilizers,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,525.8307,Fc,NaN
218707,48,Costa Rica,5062,Manure applied to Soils,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,249.7819,Fc,NaN
219012,48,Costa Rica,5063,Manure left on Pasture,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,539.4635,Fc,NaN
219317,48,Costa Rica,5064,Crop Residues,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,11.1179,Fc,NaN
219640,48,Costa Rica,5066,Burning - Crop residues,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,5.7978,Fc,NaN
219882,48,Costa Rica,6750,Net Forest conversion,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,0.0000,Fc,NaN
219942,48,Costa Rica,6751,Forestland,723113,Emissions (CO2eq) (AR5),2019,2019,3050,FAO TIER 1,kilotonnes,-7441.9840,Fc,NaN


In [26]:
em_elems = set([x for x in list(df_emissions["Element"]) if "Emissions (CO2eq) (AR5)" in x])
# check the mass balance
df_emissions_check = df_emissions[
    df_emissions["Year"].isin([2016, 2017, 2018, 2019, 2020]) & 
    df_emissions["Element"].isin(em_elems) &
    df_emissions["Item Code"].isin(dict_ic_to_sf.keys())
][["Area", "Item Code", "Item", "Year", "Value"]]

# get the mass balance to check
df_emissions_check["mb_value"] = df_emissions_check["Item Code"].replace(dict_ic_to_sf)
df_emissions_check["mb_value"] = np.array(df_emissions_check["mb_value"]) * np.array(df_emissions_check["Value"])

fields_grp = ["Area", "Year"]
fields_agg = ["mb_value"]
dict_agg = dict(zip(fields_grp, ["first" for x in fields_grp]))
dict_agg.update(dict(zip(fields_agg, ["sum" for x in fields_agg])))

df_emissions_check = df_emissions_check[fields_grp + fields_agg].groupby(fields_grp).agg(dict_agg).reset_index(drop = True)

df_emissions_check.sort_values(by = ["mb_value"], ascending = False)

,Area,Year,mb_value
23,Americas,2019,1.200969e+06
22,Americas,2018,1.198030e+06
21,Americas,2017,1.192159e+06
20,Americas,2016,1.186719e+06
404,India,2016,8.817574e+04
...,...,...,...
720,Russian Federation,2016,-7.706333e+05
959,World,2019,-1.677090e+06
957,World,2017,-1.716621e+06
958,World,2018,-1.791110e+06


In [5]:
all_items = set(df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"])
]["Item"])

all_elems = set([x for x in list(df_emissions["Element"]) if "Emissions (CO2eq) (AR5)" in x])

df_emissions_tmp = df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"]) & 
    df_emissions["Element"].isin(all_elems)
]
#df_emissions_tmp.to_csv("/Users/jsyme/Desktop/calc_fao_mb.csv", index = None, encoding = "UTF-8")

In [38]:
"""import magic

blob = open('unknown-file', 'rb').read()
m = magic.open(magic.MAGIC_MIME_ENCODING)
m.load()
encoding = m.buffer(blob)  # "utf-8" "us-ascii" etc
"""


ModuleNotFoundError: No module named 'magic'

In [37]:
fp_areas = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_All_Data_(Normalized).csv"
df_areas = pd.read_csv(
    fp_areas,
    encoding = "ascii"
)# )



UnicodeDecodeError: 'ascii' codec can't decode byte 0xe9 in position 183705: ordinal not in range(128)